In [1]:
# import the RoboNLU SDK class

from robo_sdk import RoboNLU

In [2]:
# initialize a client by setting up the NLU Service url and the necessary credentials
 
client = RoboNLU(base_endpoint="http://127.0.0.1:8000", username="user", password="bestpassword") 

In [3]:
# locally intantiate a model by generating a model object

model = client.model()

In [4]:
# by default the uuid of a model is None, which is the default needed to create a new model instance. 

model_uuid = model.get_model

print(model_uuid)

None


In [5]:
# You can interact with an existing model by passing it's uuid as a parameter in the previous method.

?client.model

Signature: client.model(model_uuid: str = None)
Docstring:
Instantiates a local model intance. If no uuid is passed as argument, the instance is void and ready to 
create a nem model instance within configured the NLU Service API.
           
arguments: - model_uuid: str, existing model_uuid string. To be used when access to existing model is required.

returns:   - a NLUResources() instantiated object.
File:      ~/anaconda3/envs/roboSDK/lib/python3.8/site-packages/robo_sdk/robo_nlu/robo_nlu.py
Type:      method


In [6]:
# Using the create model's create method, a new model instance is setup within the API.

?model.create

Signature: model.create(language: str)
Docstring:
Method to create a model object within the NLU API service.
Note that if the object's model_uuid attribute is not None, this method is not effective, and returns None.

arguments: - language: str, a 2 char code for the language that characterizes the model.

returns:   - CreateResponse() object (based upon a pydantic BaseModel).
        or
           - None,  if object's model_uuid is not None.
File:      ~/anaconda3/envs/roboSDK/lib/python3.8/site-packages/robo_sdk/robo_nlu/resources/nlu_resources.py
Type:      method


In [7]:
# This returns a Pydantic based model object, allowing for easy parsing.
 
model.create("en")

CreateResponse(timestamp='04/27/2022, 11:38:27', content=CreateContent(model_uuid='51024a12-ed09-497a-8629-237ed0c4ea1f', job_status=<Status.NEW: 'NEW'>))

In [8]:
# The uuid of the newly created instance can be retrieved accessing the object's get_model property.

model.get_model

'51024a12-ed09-497a-8629-237ed0c4ea1f'

In [9]:
# This is just a some dummy data to exemplify the train method.

data ={
  "nlu": [
    {
      "intent": "Greeting",
      "examples": [
        "Hi",
        "Hello",
        "Howdy",
        "Bonjour",
        "Ça va bien?"
      ]
    },
    {
      "intent": "Bye",
      "examples": [
        "Good bye",
        "Bye bye",
        "Au revoir",
        "C U"
      ]
    },
    {
      "lookup": "Language",
      "examples": [
        "PT",
        "ENG"
      ]
    },
    {
      "regex": "door_number",
      "examples": [
        "(\\W|^)po[#\\-]{0,1}\\s{0,1}\\d{2}[\\s-]{0,1}\\d{4}(\\W|$)"
      ]
    }
  ]
}

In [10]:
# Tou can train a model by passing loaded data in dict format as above.

train_response = model.train(data) 

In [11]:
# or train a model by referencing a nlu.yaml filepath in Rasa 3 format.

train_response = model.train("./nlu.yaml")
train_response

TrainResponse(timestamp='04/27/2022, 11:38:36', content=TrainContent(job_status=<Status.TRAINING: 'TRAINING'>))

In [12]:
# At anytime you can call the status method to know the status of the created model instance.

model.status() 

StatusResponse(timestamp='04/27/2022, 11:39:12', content=StatusContent(job_status=<Status.READY: 'READY'>))

In [13]:
# If model evaluation was requested upon training, then the resulting metrics are available by the
# metrics method when the model's status is READY.

?model.metrics

Signature: model.metrics()
Docstring:
Method to retrieve the status of model object within the NLU API service. 

arguments: No arguments are required.

returns:   - MetricsResponse() object (based upon a pydantic BaseModel).
File:      ~/anaconda3/envs/roboSDK/lib/python3.8/site-packages/robo_sdk/robo_nlu/resources/nlu_resources.py
Type:      method


In [14]:
# Predictions are made by the predict model method, obtaining a PredictResponse object as result.

predict_response_object= model.predict(json_data={"text":"Hello and goodbye"})

In [15]:
# The response from predict allows for easy content parsing

predict_response_object.__fields__

{'timestamp': ModelField(name='timestamp', type=str, required=True),
 'content': ModelField(name='content', type=PredictContent, required=True)}

In [16]:
predict_response_object.content

PredictContent(intents=[IntentResponseBody(name='Bye', confidence=0.9892255663871765), IntentResponseBody(name='Greeting', confidence=0.010774398222565651)], entities=[])

In [17]:
content = predict_response_object.content
for intent in content.intents:
    print(f"Intent '{intent.name}' predicted with {intent.confidence:.3f} confidence")

Intent 'Bye' predicted with 0.989 confidence
Intent 'Greeting' predicted with 0.011 confidence


In [18]:
predict_response_object.content.entities

[]

In [19]:
# When your done with your model, you can delete from the API

model.delete()

DeleteResponse(timestamp='04/27/2022, 11:39:33', content=DeleteContent(model_uuid='51024a12-ed09-497a-8629-237ed0c4ea1f'))